In [1]:
import os

os.chdir('../')
%pwd

'c:\\Users\\amil\\OneDrive\\Documents\\Data_Science_end_to_end_project'

In [11]:
import pandas as pd 
from dataclasses import dataclass
from pathlib import Path

from src import logger
from sklearn.model_selection import train_test_split
from src.constants import *
from src.utils.common import read_yaml,create_directories

In [12]:
@dataclass
class DataTransformationConfig:
    root_dir : Path
    data_path : str

In [20]:
class ConfigurationManger:
    def __init__(self,
                config_path= CONFIG_FILE_PATH,
                params_path= PARAMS_FILE_PATH,
                schema_path = SCHEMA_FILE_PATH
                ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self)-> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir,
            data_path= config.data_path
        )
        
        return data_transformation_config

In [23]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        
    def load_data(self)-> pd.DataFrame :
        """
        Load dataset from the configured path.
        """
        try :
            data = pd.read_csv(self.config.data_path)
            logger.info(f"Data loaded successfully from {self.config.data_path}")
            return data
        except Exception as e:
            logger.exception(e)
            raise e
    
    def check_missing_values(self,data : pd.DataFrame):
        """
        Check for missing values and log them.
        """
        missing_values = data.isnull().sum()
        logger.info(f"Missing values per column:\n{missing_values}")
        return missing_values
    
    def generate_summary_statistics(self, data: pd.DataFrame):
        """
        Generate and save summary statistics like mean, median, std, etc.
        """
        
        try :
            summary = data.describe().T
            summary_path = Path(self.config.root_dir,"summary.csv")
            summary.to_csv(summary_path)
            logger.info(f"Summary statistics saved at {summary_path}")
            return summary
        except Exception as e:
            logger.error(f"Error generating summary statistics: {e}")
            raise e
        
    def train_test_split(self, data: pd.DataFrame):
        
        train,test = train_test_split(data,test_size=0.2,random_state=42)
        
        train.to_csv(os.path.join(self.config.root_dir,"train.csv"), index = False)
        test.to_csv(os.path.join(self.config.root_dir,"test.csv"), index = False)
        
        logger.info(f"Splitted the data into training and test")
        
        logger.info(train.shape)
        logger.info(test.shape)
        
    
    def perform_full_eda(self):
        """
        Perform full EDA pipeline:
        1. Load data
        2. Check missing values
        3. Generate summary statistics
        4. Detect outliers
        5. Correlation analysis
        """
        logger.info("Starting EDA process...")
        data = self.load_data()
        
        # Step 1: Missing Values
        self.check_missing_values(data)
        
        # Step 2: Summary Statistics
        self.generate_summary_statistics(data)
        
        #step 3: train test split
        
        self.train_test_split(data)
        
        logger.info(f"completed the EDA")
        
        

In [24]:
try :
    config = ConfigurationManger()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config= data_transformation_config)
    data_transformation.perform_full_eda()
except Exception as e:
    logger.info(e)
    raise e
    

[2025-09-25 22:59:30,850:, INFO:, common:, yaml file : config\config.yaml loaded successfully]
[2025-09-25 22:59:30,905:, INFO:, common:, yaml file : params.yaml loaded successfully]
[2025-09-25 22:59:30,915:, INFO:, common:, yaml file : schema.yaml loaded successfully]
[2025-09-25 22:59:30,950:, INFO:, common:, created directory for artifacts]
[2025-09-25 22:59:30,961:, INFO:, common:, created directory for artifacts/data_transformation]
[2025-09-25 22:59:30,964:, INFO:, 1747196797:, Starting EDA process...]
[2025-09-25 22:59:31,496:, INFO:, 1747196797:, Data loaded successfully from artifacts/data_ingestion/winequality-red.csv]
[2025-09-25 22:59:31,556:, INFO:, 1747196797:, Missing values per column:
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
qu